In [1]:
import IPython
import reboting
from reboting import CouldntSaveDataException
from reboting import CsvDownloadAndParsingException
from reboting import CouldNotDownloadFileException
from reboting import CouldntCreateVisualException
from reboting import DataTooBigException
from reboting import NoVisualsPossible
import requests
import pandas as pd
import time

go over all queries for each bundesland

- for every item ask the bot with url
- and then analyze the db

In [2]:
stats = { 'total' : {
    'total' : 0,
    'success' : 0
}}
config = {
    'vienna' : '&l=http://sws.geonames.org/2761369/&l=http://sws.geonames.org/2761367/&l=http://sws.geonames.org/2761333/&l=http://sws.geonames.org/2761368/',
    'styria' : '&l=http://sws.geonames.org/2764581/',
    'lower_austria' : '&l=http://sws.geonames.org/2770542/',
    'upper_austria' : '&l=http://sws.geonames.org/2769848/',
    'tirol' : '&l=http://sws.geonames.org/2763587/&l=http://sws.geonames.org/2763586/&l=http://sws.geonames.org/2763588/',
    'vorarlberg' : '&l=http://sws.geonames.org/2762300/',
    'carinthia' : '&l=http://sws.geonames.org/2774686/',
    'burgenland' : '&l=http://sws.geonames.org/6955321/&l=http://sws.geonames.org/2781194/',
    'salzburg' : '&l=http://sws.geonames.org/2766824/&l=http://sws.geonames.org/2766818/&l=http://sws.geonames.org/2766823/&l=http://sws.geonames.org/7871495/'
}
for key, value in config.items():
    #print(key+" val: "+value)
    offset=0
    url ='http://data.wu.ac.at/odgraph/locationsearch?a=1'+value+'&offset='+str(offset)+'&limit=1'
    r = requests.get(url)
    resp = r.json()  
    stats[key]={ 'total': resp['total'], 'success' : 0}
    stats['total']['total'] = stats['total']['total']+resp['total']
    #lets start the for loop again
    #resp['total']
    for i in range(0,5):        
        offset=i        
        url ='http://data.wu.ac.at/odgraph/locationsearch?a=1'+value+'&offset='+str(offset)+'&limit=1'
        time.sleep(3)
        try: 
            r = requests.get(url)
            resp = r.json()  
            #hardcoded userid
            user_id='10213970072671093'
            searchresult = resp['results'][0]           
            geolocations = set([])
            if searchresult.get('entities',None) is not None:
                #get the array
                for entity in searchresult['entities']:
                    if(entity!=""):
                        #print("adding entity: "+entity)
                        geolocations.add(entity)              
            if searchresult.get('locations',None) is not None:
                #get the array
                for entity in searchresult['locations']:
                    if(entity!=""):
                        #print("adding entity from location: "+entity)
                        geolocations.add(entity)
            data_desc = {
                    "name" : "No name available",
                    "description" : "No description available",
                    "publisher" : "no publisher available",
                    "portal" : "No Portal available",
                    "url": searchresult['url'],
                    "user_id" : user_id,
                    "geolocations" : geolocations                    
            }
            #"temporal_end": "2014-01-09", 
            #"temporal_start": "2014-01-09", 
            if(searchresult.get('temporal_end',None) is not None and searchresult.get('temporal_start',None) is not None):
                print("adding temporal parts")
                data_desc['temporal_end'] = searchresult['temporal_end']
                data_desc['temporal_start'] = searchresult['temporal_start']
            if searchresult.get('dataset',None) is not None:
                if searchresult['dataset'].get('dataset_name',None) is not None:
                    data_desc['name'] = searchresult['dataset']['dataset_name']
                if searchresult['dataset'].get('dataset_description',None) is not None:
                    data_desc['description'] = searchresult['dataset']['dataset_description']
                if searchresult['dataset'].get('publisher',None) is not None:
                    data_desc['publisher'] = searchresult['dataset']['publisher']
            if searchresult.get('portal',None) is not None:
                data_desc['portal'] = searchresult['portal']            
            try:
                response = reboting.importAsNew(data_desc = data_desc)
                #url = 'https://doh.23degrees.io/embed/'+slug
                print(response)
                if(response>0):                    
                    stats[key]['success'] = stats[key]['success']+1
                    stats['total']['success'] = stats['total']['success']+1                    
            except CouldntSaveDataException as e:
                print("ERROR: couldnt save data on 23 degree server")
            except CsvDownloadAndParsingException as e:
                print("ERROR: something went wrong when downloading the file")
            except CouldNotDownloadFileException as e:
                print("ERROR: couldnt download file")
            except CouldntCreateVisualException as e:
                print("ERROR: couldnt create visual")
            except DataTooBigException as e:
                print("ERROR: too many lines: "+str(e))   
            except NoVisualsPossible as e:
                print("ERROR: no visual possible: "+str(e))
            print(str(i)+" success: "+str(stats['total']['success']))
        except Exception as e:
            raise Exception(e)
            print("new error: "+str(e))

adding temporal parts
201
135
0 success: 1
adding temporal parts
201
88
1 success: 2
adding temporal parts
201
88
2 success: 3
adding temporal parts
201
39
3 success: 4
adding temporal parts
201
270
4 success: 5
adding temporal parts
201
15
0 success: 6
201
30
1 success: 7
adding temporal parts
201
4
2 success: 8
adding temporal parts
201
12
3 success: 9
adding temporal parts
201
24
4 success: 10
201
21
0 success: 11
201
2
1 success: 12
ERROR: something went wrong when downloading the file
2 success: 12
ERROR: something went wrong when downloading the file
3 success: 12
ERROR: no visual possible: http://open-data.noe.gv.at/ogd-data/RU2/noe_deaths_sex_2011_2016_lau2.csv
4 success: 12
201
161
0 success: 13
adding temporal parts
201
56
1 success: 14
201
2
2 success: 15
201
3
3 success: 16
adding temporal parts
201
21
4 success: 17
adding temporal parts
201
21
0 success: 18
adding temporal parts
201
9
1 success: 19
adding temporal parts
201
24
2 success: 20
adding temporal parts
201
33
3 s

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2802: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


400
0
4 success: 26
adding temporal parts
201
9
0 success: 27
adding temporal parts
201
16
1 success: 28
adding temporal parts
201
12
2 success: 29
adding temporal parts
201
9
3 success: 30
adding temporal parts
201
22
4 success: 31
adding temporal parts
201
88
0 success: 32
201
1
1 success: 33
adding temporal parts
524
0
2 success: 33
201
184
3 success: 34
adding temporal parts
201
95
4 success: 35
adding temporal parts
201
14
0 success: 36
adding temporal parts
201
1
1 success: 37
201
6
2 success: 38
ERROR: no visual possible: http://www.salzburg.gv.at/ogd/e7f97934-ab43-4024-abc5-e5b2a37ea215/migration-szg-sex-nat-district.csv
3 success: 38
ERROR: no visual possible: http://www.salzburg.gv.at/ogd/f19ef3e9-423e-4856-ae43-acf9ccd8cfd8/marriage-szg-sex-age-district.csv
4 success: 38


In [ ]:
stats